In [1]:
from sympy import *
from sympy import sin, pde_separate
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector.printing import vlatex, vpprint
import numpy as np

import matplotlib.pyplot as plt
init_printing(use_latex='mathjax',use_unicode=True)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
q1, q2, q3, q4, q5, q6 = dynamicsymbols('q1 q2 q3 q4 q5 q6')
dq1, dq2, dq3, dq4, dq5, dq6 = symbols('dq1 dq2 dq3 dq4 dq5 dq6')
ddq1, ddq2, ddq3, ddq4, ddq5, ddq6 = symbols('ddq1 ddq2 ddq3 ddq4 ddq5 ddq6')
Q1, Q2, Q3, Q4, Q5, Q6 = symbols('Q1 Q2 Q3 Q4 Q5 Q6')
m, Ixx, Iyy, Izz, g, l = symbols('m Ixx Iyy Izz g l', constant = True)
F1, F2, F3, F4 = symbols('F1 F2 F3 F4')
q_3, q_5, q_6 = symbols('q_3 q_5 q_6')

In [3]:
q = Matrix([q1, q2, q3, q4, q5, q6])
dq = diff(q, Symbol('t'))
ddq = diff(dq, Symbol('t'))


$
T(q)=\frac{1}{2}\dot{q}^{T}M(q)*\dot{q}
$

where M(q) is defined as:

$
M(q)=J_{v}(q)^{T}m_{rr}J_{v}(q)+J_{w}(q)^{T}m_{\theta\theta}J_{w}(q)
$

In [4]:
# J_w = Matrix([[0,0,0,-sin(q[4]), 0, 1],
#              [0,0,0, cos(q[4])*sin(q[5]), cos(q[5]), 0],
#              [0,0,0, cos(q[4])*cos(q[5]), sin(q[5]), 0]])
# J_v = Matrix([[1,0,0,0,0,0],
#              [0,1,0,0,0,0],
#              [0,0,1,0,0,0]])
# m_rr = eye(3)*m
# m_tt = Matrix([[Ixx, 0, 0],
#               [0, Iyy, 0],
#               [0, 0, Izz]])
# M_q = J_v.T * m_rr * J_v + J_w.T * m_tt * J_w
# M_q = trigsimp(M_q)

# T = trigsimp(simplify(trigsimp(0.5*dq.T*M_q*dq)))
# T

T = [0.5*m*dq[0]**2 + 0.5*m*dq[1]**2 + 0.5*m*dq[2]**2 + 0.5*Ixx*dq[5]**2 + 0.5*Iyy*dq[4]**2 + 0.5*Izz*dq[3]**2 - (Izz-Iyy)*dq[3]*dq[4]*cos(q[4])*cos(q[5])*sin(q[5]) - Ixx*dq[3]*dq[5]*sin(q[4])]


In [6]:
V = -m*g*q[2]

L = T[0]-V
L = collect(L, [dq[3]**2, dq[4]**2])

L = L.trigsimp(method = 'fu')
L

                                                          2                   
                 d         d                   ⎛d        ⎞            ⎛d      
- Ixx⋅sin(q₅(t))⋅──(q₄(t))⋅──(q₆(t)) + 0.5⋅Ixx⋅⎜──(q₆(t))⎟  + 0.5⋅Iyy⋅⎜──(q₅(t
                 dt        dt                  ⎝dt       ⎠            ⎝dt     

   2                      2                                2                  
  ⎞            ⎛d        ⎞                      ⎛d        ⎞          ⎛d       
))⎟  + 0.5⋅Izz⋅⎜──(q₄(t))⎟  + g⋅m⋅q₃(t) + 0.5⋅m⋅⎜──(q₁(t))⎟  + 0.5⋅m⋅⎜──(q₂(t)
  ⎠            ⎝dt       ⎠                      ⎝dt       ⎠          ⎝dt      

  2                    2                                                      
 ⎞          ⎛d        ⎞                                                  d    
)⎟  + 0.5⋅m⋅⎜──(q₃(t))⎟  - (-Iyy + Izz)⋅sin(q₆(t))⋅cos(q₅(t))⋅cos(q₆(t))⋅──(q₄
 ⎠          ⎝dt       ⎠                                                  dt   

              
     d        
(t))⋅──(q₅(t))
    

The L above is with no trig simplification, actually it is expanded.

In the next cell you are looking at L but simplified with trig by hand.

In [7]:
L = 0.5*m*dq[0]**2 + 0.5*m*dq[1]**2 + 0.5*m*dq[2]**2 +0.5*Ixx*dq[5]**2 + 0.5*Iyy*dq[4]**2 + 0.5*Izz*dq[3]**2 - (Izz-Iyy)*dq[3]*dq[4]*cos(q[4])*cos(q[5])*sin(q[5]) - Ixx*dq[3]*dq[5]*sin(q[4])

Euler-Lagrange equations of motions from the following equation:

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{j}}})-\frac{\partial\mathcal{L}(q)}{\partial q_{j}}=Q_{j}(q) =\overrightarrow{F}_{R}\frac{\partial\overrightarrow{r_{G}}}{\partial q_{i}}+\overrightarrow{M}_{G}\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{j}}}, i=1:6, j=4:6
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{1}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{1}}= (\sum_{i=1}^{4}F_{i} - mg)\frac{\partial\overrightarrow{r_{G}}}{\partial q_{1}}+(\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}))\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{1}}}
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{2}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{2}}= (\sum_{i=1}^{4}F_{i} - mg)\frac{\partial\overrightarrow{r_{G}}}{\partial q_{2}}+(\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}))\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{2}}}
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{3}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{3}}= (\sum_{i=1}^{4}F_{i} - mg)\frac{\partial\overrightarrow{r_{G}}}{\partial q_{3}}+(\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}))\frac{\partial\overrightarrow{\omega}}{\partial
\dot{q_{3}}}
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{4}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{4}}= (\sum_{i=1}^{4}F_{i} - mg)\frac{\partial\overrightarrow{r_{G}}}{\partial q_{4}}+(\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}))\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{4}}}
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{5}}})- \frac{\partial\mathcal{L}(q)}{\partial q_{5}}= (\sum_{i=1}^{4}F_{i} - mg)\frac{\partial\overrightarrow{r_{G}}}{\partial q_{5}}+(\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}))\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{5}}}
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{6}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{6}}= (\sum_{i=1}^{4}F_{i} - mg)\frac{\partial\overrightarrow{r_{G}}}{\partial q_{6}}+(\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}))\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{6}}}
$

$
(\sum_{i=1}^{4}F_{i} - mg)\frac{\partial\overrightarrow{r_{G}}}{\partial q_{1}} =\left[\begin{array}{c} 0 \\ 0 \\ F_{1} + F_{2} + F_{3} + F_{4} -mg\end{array}\right]\left[\begin{array}{c} 1 \\ 0 \\ 0\end{array}\right] = 0
$

$
(\sum_{i=1}^{4}F_{i} - mg)\frac{\partial\overrightarrow{r_{G}}}{\partial q_{2}} =\left[\begin{array}{c} 0 \\ 0 \\ F_{1} + F_{2} + F_{3} + F_{4} -mg\end{array}\right]\left[\begin{array}{c} 0 \\ 1 \\ 0\end{array}\right] = 0
$

$
(\sum_{i=1}^{4}F_{i} - mg)\frac{\partial\overrightarrow{r_{G}}}{\partial q_{3}} =\left[\begin{array}{c} 0 \\ 0 \\ F_{1} + F_{2} + F_{3} + F_{4} -mg\end{array}\right]\left[\begin{array}{c} 0 \\ 0 \\ 1\end{array}\right] = F_{1} + F_{2} + F_{3} + F_{4} -mg
$


Assuming the x axis lies along the structure towards F1, and y axis lies along the structure towards F4. As shown in the drone_diag.

Also assuming that the length from G to each force is symmetric, shown as l.

$
\overrightarrow{r_1} = \left[\begin{array}{c} l \\ 0 \\ 0\end{array}\right],
\overrightarrow{r_2} = \left[\begin{array}{c} 0 \\ -l \\ 0\end{array}\right],
\overrightarrow{r_3} = \left[\begin{array}{c} -l \\ 0 \\ 0\end{array}\right],
\overrightarrow{r_4} = \left[\begin{array}{c} 0 \\ l \\ 0\end{array}\right]
$

$
\overrightarrow{F_1} = \left[\begin{array}{c} 0 \\ 0 \\ F_1 \end{array}\right],
\overrightarrow{F_2} = \left[\begin{array}{c} 0 \\ 0 \\ F_2 \end{array}\right],
\overrightarrow{F_3} = \left[\begin{array}{c} 0 \\ 0 \\ F_3 \end{array}\right],
\overrightarrow{F_4} = \left[\begin{array}{c} 0 \\ 0 \\ F_4 \end{array}\right]
$

$
\overrightarrow{r_1} X \overrightarrow{F_1} = \left[\begin{array}{c} 0 \\ -F_1 l \\ 0 \end{array}\right],
\overrightarrow{r_2} X \overrightarrow{F_2} = \left[\begin{array}{c} -F_2 l \\ 0 \\ 0 \end{array}\right],
\overrightarrow{r_3} X \overrightarrow{F_3} = \left[\begin{array}{c} 0 \\ F_3 l \\ 0 \end{array}\right],
\overrightarrow{r_4} X \overrightarrow{F_4} = \left[\begin{array}{c} F_4 l \\ 0 \\ 0 \end{array}\right]
$

$
\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}) =
\left[\begin{array}{c} F_4 l -F_2 l \\ F_3 l - F_1 l \\ 0\end{array}\right]
$

$
\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{1}}} = 0, \frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{2}}} = 0, \frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{3}}} = 0 
$

$
\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{4}}} = \left[\begin{array}{c} -\sin{\theta} \\ \cos{\theta}\sin{\psi} \\ \cos{\theta}\cos{\psi} \end{array}\right],
\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{5}}} = \left[\begin{array}{c} 0 \\ \cos{\psi} \\ \sin{\psi} \end{array}\right],
\frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{6}}} = \left[\begin{array}{c} 1 \\ 0 \\ 0\end{array}\right]
$

$
\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}) \frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{4}}} =
\left[\begin{array}{c} F_4 l -F_2 l \\ F_3 l - F_1 l \\ 0\end{array}\right]^{T} \left[\begin{array}{c} -\sin{\theta} \\ \cos{\theta}\sin{\psi} \\ \cos{\theta}\cos{\psi} \end{array}\right]
$

$
\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}) \frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{5}}} =
\left[\begin{array}{c} F_4 l -F_2 l \\ F_3 l - F_1 l \\ 0\end{array}\right]^{T} \left[\begin{array}{c} 0 \\ \cos{\psi} \\ \sin{\psi} \end{array}\right]
$

$
\sum_{i=1}^{4}(\overrightarrow{r_{i}}\times\overrightarrow{F_{i}}) \frac{\partial\overrightarrow{\omega}}{\partial\dot{q_{6}}} =
\left[\begin{array}{c} F_4 l -F_2 l \\ F_3 l - F_1 l \\ 0\end{array}\right]^{T} \left[\begin{array}{c} 1 \\ 0 \\ 0\end{array}\right]
$


THEN:

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{1}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{1}}= 0
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{2}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{2}}= 0
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{3}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{3}}= F_{1} + F_{2} + F_{3} + F_{4} -mg
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{4}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{4}}= \left[\begin{array}{c} F_4 l -F_2 l \\ F_3 l - F_1 l \\ 0\end{array}\right]^{T} \left[\begin{array}{c} -\sin{\theta} \\ \cos{\theta}\sin{\psi} \\ \cos{\theta}\cos{\psi} \end{array}\right]
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{5}}})- \frac{\partial\mathcal{L}(q)}{\partial q_{5}}= \left[\begin{array}{c} F_4 l -F_2 l \\ F_3 l - F_1 l \\ 0\end{array}\right]^{T} \left[\begin{array}{c} 0 \\ \cos{\psi} \\ \sin{\psi} \end{array}\right]
$

$
\frac{d}{dt}(\frac{\partial\mathcal{L}(q)}{\partial\dot{q_{6}}}) - \frac{\partial\mathcal{L}(q)}{\partial q_{6}}= \left[\begin{array}{c} F_4 l -F_2 l \\ F_3 l - F_1 l \\ 0\end{array}\right]^{T} \left[\begin{array}{c} 1 \\ 0 \\ 0\end{array}\right]
$

In [8]:
vF1 = Matrix([0,0,F4])
vr1 = Matrix([0,l,0])

vr1.cross(vF1)

⎡F₄⋅l⎤
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎣ 0  ⎦

In [9]:
# d/dt(dL/d(dq)) - dL/dq = Q
dl1 = diff(diff(L, dq[0]), Symbol('t'))
dl2 = diff(diff(L, dq[1]), Symbol('t'))
dl3 = diff(diff(L, dq[2]), Symbol('t'))
dl4 = diff(diff(L, dq[3]), Symbol('t'))
dl5 = diff(diff(L, dq[4]), Symbol('t'))
dl6 = diff(diff(L, dq[5]), Symbol('t'))

l1 = diff(L, q[0])
l2 = diff(L, q[1])
l3 = diff(L, q[2])
l4 = diff(L, q[3])
l5 = diff(L, q[4])
l6 = diff(L, q[5])

EL1 = dl1 - l1 - Q1
EL2 = dl2 - l2 - Q2
EL3 = dl3 - l3 - Q3
EL4 = dl4 - l4 - Q4
EL5 = dl5 - l5 - Q5
EL6 = dl6 - l6 - Q6

# EL6.coeff(ddq[5])

eqs = (EL1, EL2, EL3, EL4, EL5, EL6)
variables = (ddq[0], ddq[1], ddq[2], ddq[3], ddq[4], ddq[5])
EL4
EL5
EL6

                   2                                                         2
                  d                          d         d                    d 
- Ixx⋅sin(q₅(t))⋅───(q₆(t)) - Ixx⋅cos(q₅(t))⋅──(q₅(t))⋅──(q₆(t)) + 1.0⋅Izz⋅───
                   2                         dt        dt                    2
                 dt                                                        dt 

                                                                       2      
                                                            ⎛d        ⎞       
(q₄(t)) - Q₄ - (Iyy - Izz)⋅sin(q₅(t))⋅sin(q₆(t))⋅cos(q₆(t))⋅⎜──(q₅(t))⎟  - (Iy
                                                            ⎝dt       ⎠       
                                                                              

                                                                              
            2                   d         d                                   
y - Izz)⋅sin (q₆(t))⋅cos(q₅(t))⋅──(q₅(t))⋅──(q₆(t)

                                               2                              
               d         d                    d                               
Ixx⋅cos(q₅(t))⋅──(q₄(t))⋅──(q₆(t)) + 1.0⋅Iyy⋅───(q₅(t)) - Q₅ - (Iyy - Izz)⋅sin
               dt        dt                    2                              
                                             dt                               

                                                                              
2                   d         d                                               
 (q₆(t))⋅cos(q₅(t))⋅──(q₄(t))⋅──(q₆(t)) + (Iyy - Izz)⋅sin(q₆(t))⋅cos(q₅(t))⋅co
                    dt        dt                                              
                                                                              

           2                                                                
          d                                     2        d         d        
s(q₆(t))⋅───(q₄(t)) + (Iyy - Izz)⋅cos(q₅(t))⋅cos (q₆(t

                   2                                                         2
                  d                          d         d                    d 
- Ixx⋅sin(q₅(t))⋅───(q₄(t)) - Ixx⋅cos(q₅(t))⋅──(q₄(t))⋅──(q₅(t)) + 1.0⋅Ixx⋅───
                   2                         dt        dt                    2
                 dt                                                        dt 

                                                                              
                              2                   d         d                 
(q₆(t)) - Q₆ + (Iyy - Izz)⋅sin (q₆(t))⋅cos(q₅(t))⋅──(q₄(t))⋅──(q₅(t)) - (Iyy -
                                                  dt        dt                
                                                                              

                                                
                    2        d         d        
 Izz)⋅cos(q₅(t))⋅cos (q₆(t))⋅──(q₄(t))⋅──(q₅(t))
                             dt        dt       
            

In [10]:
#After hand simplifying this is rewritten 
EL4 = -Ixx*sin(q[4])*ddq[5] - Ixx*cos(q[4])*dq[4]*dq[5] + Izz*ddq[3] - Q4 - (Ixx-Izz)*sin(q[4])*sin(q[5])*cos(q[5])*ddq[4]**2 + (Iyy-Izz)*sin(q[5])*cos(q[4])*cos(q[5])*ddq[4]

EL5 = Ixx*cos(q[4])*dq[3]*dq[5] + Iyy*ddq[4] - Q5 + (Iyy-Izz)*sin(q[5])*cos(q[4])*cos(q[5])*ddq[3]

EL6 = -Ixx*sin(q[4])*ddq[3] - Ixx*cos(q[4])*dq[3]*dq[4] + Ixx*ddq[5] - Q6

eqs = (EL1, EL2, EL3, EL4, EL5, EL6)
variables = (ddq[0], ddq[1], ddq[2], ddq[3], ddq[4], ddq[5])
print(vlatex(EL1))


- Q_{1} + 1.0 m \ddot{q}_{1}


In [ ]:
#This is the main function to seperate the double derivative

F = solve(eqs, variables)

In [15]:
F

⎡⎛                    ⎛                                                       
⎢⎜                    ⎜          2                   2           2           2
⎢⎜            0.03125⋅⎜- 32.0⋅Ixx ⋅Iyy⋅sin(q₅(t))⋅sin (q₆(t))⋅cos (q₅(t))⋅cos 
⎢⎜Q₁  Q₂  Q₃          ⎝                                                       
⎢⎜──, ──, ──, ────────────────────────────────────────────────────────────────
⎢⎜m   m   m                                                                   
⎣⎝                                                                            

                                                                              
        d         d                   2                   2           2       
(q₆(t))⋅──(q₄(t))⋅──(q₆(t)) + 32.0⋅Ixx ⋅Izz⋅sin(q₅(t))⋅sin (q₆(t))⋅cos (q₅(t))
        dt        dt                                                          
──────────────────────────────────────────────────────────────────────────────
                                                   

# $\dot{x} = F(q) + G(q)Q$

In [14]:
#This breaks out the 6 equations all f(Q)

F1 = F[ddq[0]]
F2 = F[ddq[1]]
F3 = F[ddq[2]]
F4 = F[ddq[3]]
F5 = F[ddq[4]]
F6 = F[ddq[5]]

# print(vlatex(F3))
simplify(F3)
F2
F3
F5
F6

TypeError: list indices must be integers or slices, not Derivative

In [ ]:
e1 = -ddq1 + (1/m)*(F1+F2+F3+F4)(cos(q[3])*sin(q[4])*cos(q[5]) + sin(q[3]*sin(q[5])))
e2 = -ddq2 + (1/m)*(F1+F2+F3+F4)(sin(q[3])*sin(q[4])*cos(q[5]) - cos(q[3]*sin(q[5])))
e3 = -ddq3 + (1/m)*(F1+F2+F3+F4)*cos(q[4])*cos(q[5]) - (W/m)
e4 = a*(F4-F2)*sin(q[4]) + a*(F3-F1)*cos(q[4])*sin(q[5])



In [30]:
G = zeros(12,4)

#Dummy lists for the loop
_f = [F1,F2,F3,F4,F5,F6]
_q = [Q1,Q2,Q3,Q4,Q5,Q6]


for n in range(0,3):
    
    G[6+n,n] = apart(_f[n+3],_q[n+3]).args[1]
    
G[0,0] = apart(F1,Q4).args[1]
G[0,1] = apart(F1,Q5).args[1]
G[0,2] = apart(F1,Q3).args[1]
G[0,2] = apart(F1,Q4).args[1]

# G[9,10] = apart(F4,Q5).args[1]
# G[9,11] = apart(F4,Q6).args[1]

# G[10,9] = apart(F5,Q4).args[1]
# G[10,11] = apart(F5,Q6).args[1]

# G[11,9] = apart(F6,Q4).args[1]
# G[11,10] = apart(F6,Q5).args[1]

    

In [31]:
#Workup G from the "apart" function and args[1]

G

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [ ]:
def funcs(t,z):
    q1, q2, q3, q4, q5, q6 = z
    F1.subs(q[4],q4)
    return [F1, F2, F3, F4, F5, F6]

In [ ]:
help(diff)